<p style="font-size: 18px;">
  This is the accompanying code for the post titled "From Pixels to Patterns: Exploring CNN Architectures and Image Recognition using PyTorch"<br>
  You can find it <a href="https://pureai.substack.com/p/from-pixels-to-patterns">here</a>.<br>
  Published: November 25, 2023<br>
  <a href="https://pureai.substack.com">https://pureai.substack.com</a>
</p>

Welcome to this Jupyter notebook! If you're new to Python or don't have it installed on your system, don't worry; you can still follow along and explore the code.

Here's a quick guide to getting started:

- Using an Online Platform: You can run this notebook in a web browser using platforms like Google Colab or Binder. These services offer free access to Jupyter notebooks and don't require any installation.
- Installing Python Locally: If you'd prefer to run this notebook on your own machine, you'll need to install Python. A popular distribution for scientific computing is Anaconda, which includes Python, Jupyter, and other useful tools.
  - Download Anaconda from [here](https://www.anaconda.com/download).
  - Follow the installation instructions for your operating system.
  - Launch Jupyter Notebook from Anaconda Navigator or by typing jupyter notebook in your command line or terminal.
- Opening the Notebook: Once you have Jupyter running, navigate to the location of this notebook file (.ipynb) and click on it to open.
- Running the Code: You can run each cell in the notebook by selecting it and pressing Shift + Enter. Feel free to modify the code and experiment with it.
- Need More Help?: If you're new to Python or Jupyter notebooks, you might find these resources helpful:
  - [Python.org's Beginner's Guide](https://docs.python.org/3/tutorial/index.html)
  - [Jupyter Notebook Basics](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html)

Happy coding, and enjoy exploring the fascinating world of CNNs with PyTorch!

### Building a Basic CNN Architecture with PyTorch

In this section, we'll walk through how to build a basic Convolutional Neural Network (CNN) using PyTorch. We'll create a simple model suitable for classifying images. Before diving into the code, ensure you have PyTorch installed in your environment.

#### Setting Up

First, import the necessary libraries:

In [ ]:
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np

Next, let's perform a check to utilize the GPU if it's available:

In [ ]:
# Check if GPU is available and set the device accordingly 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()
print(f'Using device: {torch.cuda.get_device_name(0)}')

#### Preparing the Dataset

We'll use a standard dataset like MNIST, which consists of handwritten digits. PyTorch makes it easy to load and preprocess datasets:

In [ ]:
# Transformations applied on each image
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalizing the images
])

# Loading the training dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Loading the test dataset
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Let's view a few of the training images to understand what we're working with: 

In [ ]:
# Create a function to plot images in a grid
def show_images_grid(images, labels, nrows=3, ncols=3):
    fig, axes = plt.subplots(nrows, ncols, figsize=(5, 5))
    axes = axes.flatten()
    for img, label, ax in zip(images, labels, axes):
        if img.shape[0] == 1:
            img = img.squeeze()
        else:
            img = torchvision.transforms.functional.to_pil_image(img)
        
        ax.imshow(img, cmap='gray')
        ax.axis('off')
        ax.set_title(f'Label: {label.item()}')
    plt.tight_layout()
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)

images = images[:9]
labels = labels[:9]

show_images_grid(images, labels, nrows=3, ncols=3)

#### Defining the CNN Architecture

Now, let's define our CNN architecture. We'll create a simple network with two convolutional layers followed by two fully connected layers:

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(7*7*64, 128)
        self.fc2 = nn.Linear(128, 10)

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 7*7*64)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)
print(model)

#### Model Training

Next, we'll define our loss function and optimizer, and train the model over 5 epochs:

In [ ]:
%%time

# Define the network parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):  # 5 epochs
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)  # Move data to the device

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            # Calculate accuracy on the training set
            acc = accuracy_score(labels.cpu().tolist(), np.argmax(outputs.cpu(), axis=1).tolist())
        sys.stderr.write(f"\rEpoch {epoch+1:02d}/{5:02d}, Batch {i+1:02d} | Loss: {loss.item():<6.2f} | Tr Acc: {acc*100:3.2f}%")
        sys.stderr.flush()

#### Test Model Performance

In [ ]:
# Ensuring the model is in evaluation mode
model.eval()

y_test, y_pred = [], []
for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    y_test.extend(labels.cpu().numpy())

    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        y_pred.extend(predicted.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the model on the test images: {accuracy * 100:.2f}%')

#### View Predicted Images

In [ ]:
# View a subset of the predicted images
fig = plt.figure(figsize=(10, 10))
columns = 4
rows = 5
for i in range(1, columns*rows + 1):
    img_xy = np.random.randint(len(test_dataset))
    img = test_dataset[img_xy][0][0,:,:]
    fig.add_subplot(rows, columns, i)
    plt.title(f'Predicted: {y_pred[img_xy]}, Label: {y_test[img_xy]}')
    plt.axis('off')
    plt.imshow(img, cmap='gray')
plt.show()